In [1]:
from collections import OrderedDict
from concurrent.futures import ThreadPoolExecutor, as_completed

import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Download the data

In [2]:
from collections import defaultdict


api = wandb.Api()
target_names = ["gaussian_mixture40", "gaussian_mixture40", "funnel", "many_well"]
dims = [2, 5, 10, 32]

algorithm_names = [
    "dds",
    "lv",
    "tb",
    "gfniw",
    "gfnsmc_flowdim64",
    "gfnbuf",
    "gfnrbuf",
    "gfnlbuf",
    "gfnuiwbuf",
    "gfnpiwbuf",
    "gfnsmcbuf_flowdim64",
    "gfnsmcpiwbuf_flowdim64",
]

wandb_tag_filter = {
    "$all": ["final", "main_wo_lp"],
    "$nin": ["hidden", "legacy"],
}

wandb_filter = {
    "tags": wandb_tag_filter,
    "config.algorithm_model_use_lp": False,
}

runs = defaultdict(dict)

# first key is target_name-dim
for target_name, dim in zip(target_names, dims):
    wandb_filter["config.target_name"] = target_name
    wandb_filter["config.target_dim"] = dim

    for algorithm_name in algorithm_names:
        wandb_filter["config.wandb_name"] = algorithm_name
        runs[f"{target_name}-{dim}d"][algorithm_name] = api.runs(f"sanghyeok-choi/sampling_bench", filters=wandb_filter)
        print(f"Number of runs in {target_name}-{dim}d/{algorithm_name}: {len(runs[f'{target_name}-{dim}d'][algorithm_name])}")


Number of runs in gaussian_mixture40-2d/dds: 5
Number of runs in gaussian_mixture40-2d/lv: 5
Number of runs in gaussian_mixture40-2d/tb: 5
Number of runs in gaussian_mixture40-2d/gfniw: 5
Number of runs in gaussian_mixture40-2d/gfnsmc_flowdim64: 5
Number of runs in gaussian_mixture40-2d/gfnbuf: 5
Number of runs in gaussian_mixture40-2d/gfnrbuf: 5
Number of runs in gaussian_mixture40-2d/gfnlbuf: 5
Number of runs in gaussian_mixture40-2d/gfnuiwbuf: 5
Number of runs in gaussian_mixture40-2d/gfnpiwbuf: 5
Number of runs in gaussian_mixture40-2d/gfnsmcbuf_flowdim64: 5
Number of runs in gaussian_mixture40-2d/gfnsmcpiwbuf_flowdim64: 5
Number of runs in gaussian_mixture40-5d/dds: 5
Number of runs in gaussian_mixture40-5d/lv: 5
Number of runs in gaussian_mixture40-5d/tb: 5
Number of runs in gaussian_mixture40-5d/gfniw: 5
Number of runs in gaussian_mixture40-5d/gfnsmc_flowdim64: 5
Number of runs in gaussian_mixture40-5d/gfnbuf: 5
Number of runs in gaussian_mixture40-5d/gfnrbuf: 5
Number of runs i

In [3]:
### Prepare dataframes

# Prepare metrics (columns)
metrics = ["KL/eubo", "KL/elbo", "logZ/reverse", "discrepancies/sd", "KL/eubo-elbo"]
metrics_std = [f"{m}_std" for m in metrics]

# make a dataframe with group_keys as multi-index and metrics as columns
metrics_dfs = {}
for target_name_dim in runs.keys():
    # row is algorithm_name
    # column is metrics_columns
    metrics_dfs[target_name_dim] = pd.DataFrame(columns=metrics + metrics_std) 

In [4]:

def process_group_key(alg_name, alg_runs, metrics, timesteps, n_seeds):
    # Save to numpy array first
    metrics_arr = np.zeros((n_seeds, len(metrics)))

    metrics_wo_gap = [m for m in metrics if m != "KL/eubo-elbo"]
    for i, run in enumerate(alg_runs):
        # Fetch last 5 metrics
        last_5_df = run.history(samples=timesteps[-1] + 1, keys=metrics_wo_gap)
        last_5_df.set_index("_step", inplace=True)
        last_5_values = last_5_df.loc[timesteps, metrics_wo_gap]
        last_5_values.loc[:, "KL/eubo-elbo"] = last_5_values["KL/eubo"] - last_5_values["KL/elbo"]
        metrics_arr[i] = last_5_values.values.mean(axis=0)

    return {
        'key': alg_name,
        'mean': metrics_arr.mean(axis=0),  # average over 5 seeds
        'std': metrics_arr.std(axis=0),  # std over 5 seeds
    }


last_5_timesteps = [19200, 19400, 19600, 19800, 19999]  # to be averaged
n_seeds = 5

for target_name_dim in runs.keys():
    print(f"Downloading runs for {target_name_dim}")

    target_runs = runs[target_name_dim]
    keys = target_runs.keys()
    with ThreadPoolExecutor(max_workers=min(32, len(runs[target_name_dim]))) as executor:
        futures = {
            executor.submit(
                process_group_key, 
                key,
                target_runs[key],
                metrics,
                last_5_timesteps,
                n_seeds,
            ): key for key in keys
        }

        for future in as_completed(futures):
            result = future.result()
            key = result['key']

            # Store results in dataframes
            metrics_dfs[target_name_dim].loc[key, metrics] = result['mean']
            metrics_dfs[target_name_dim].loc[key, metrics_std] = result['std']

### Main Results

In [5]:
def print_latex_table(
    target_names: list[str],
    target_names_to_display_name: dict[str, str],
    metrics: list[str],
    metric_names: list[str],
    final_metrics_dfs: dict[str, pd.DataFrame],
):
    # header = f"{'Algorithm': <150}"
    # for target_name in target_names:
    #     name, dim = target_name.replace('_', '').split('-')
    #     for metric_name in metric_names:
    #         col = f"{name.upper()}({dim}) {metric_name}"
    #         header += f" & {col: <24}"
    # header += "\\\\"
    # print(header)

    # Print latex table with column: target_names[0]-elbo & target_names[0]-eubo & target_names[1]-elbo & energy_names[1]-eubo & ...
    indices = final_metrics_dfs[target_names[0]].index
    for df in final_metrics_dfs.values():
        if not np.all(df.index == indices):
            raise ValueError("All dataframes must have the same index")

    out = f"{'Algorithm': <20}"
    for target_name in target_names:
        for metric_name in metric_names:
            col = f"{target_names_to_display_name[target_name]} {metric_name}"
            out += f" & {col: <24}"
    out += "\n"

    for idx in indices:
        out += f"{idx: <20}\n"
        for target_name in target_names:
            temp_df = final_metrics_dfs[target_name].loc[idx]
            temp_df = final_metrics_dfs[target_name].loc[idx]
            for metric in metrics:
                val, std = temp_df[metric], temp_df[f"{metric}_std"]
                if val < -1e4 or val > 1e4:
                    # Convert to scientific notation
                    record = f"{val:.2e}\scriptsize$\pm${std:0.2e} \n"
                else:
                    record = f"{val:0.2f}\scriptsize$\pm${std:0.2f} \n"
                out += f"\t& {record: <24}"
        out += "\\\\"
        out += "\n"
    print(out)
    

analysis_name = "main"
lp = [False]

report_metrics = ["KL/elbo", "KL/eubo", "discrepancies/sd"]
report_metric_names = ["ELBO", "EUBO", "Sinkhorn"]
# metrics = ["KL/eubo-elbo", "discrepancies/sd"]
# metric_names = ["EUBO-ELBO", "Sinkhorn"]

run_name_to_display_name = OrderedDict({
    "dds": "DDS",
    "lv": "LogVar",
    "tb": "TB",
    # "gfniw": "TB + IW",
    # "gfnsmc": "TB + SMC",
    "gfnsmc_flowdim64": "TB + SMC",
    "gfnbuf": "TB + Buffer",
    # "gfnrbuf": "TB + R-Buffer",
    # "gfnlbuf": "TB + L-Buffer",
    # "gfnuiwbuf": "TB + UIW-Buffer",
    "gfnpiwbuf": "TB + IW-Buffer",
    # "gfnsmcbuf": "TB + SMC + Buffer",
    # "gfnsmcbuf_flowdim64": "TB + SMC + Buffer",
    # "gfnsmcpiwbuf": "TB + SMC + IW-Buffer",
    "gfnsmcpiwbuf_flowdim64": "TB + SMC + IW-Buffer",
})

target_names_to_display_name = {
    "gaussian_mixture40-2d": "GMM40 (2d)",
    "gaussian_mixture40-5d": "GMM40 (5d)",
    "funnel-10d": "Funnel (10d)",
    "many_well-32d": "Many Well (32d)",
}

new_dfs = {key: df.copy() for key, df in metrics_dfs.items()}
for key, df in new_dfs.items():
    df.index = df.index.map(run_name_to_display_name)
    # Filter out keys that are not in run_name_to_display_name
    df = df[df.index.isin(list(run_name_to_display_name.values()))]
    df = df.reindex(list(run_name_to_display_name.values()))
    new_dfs[key] = df

# save_final_metrics_dfs_to_csv(energy_names, metrics, metric_names, new_df, analysis_name)
print_latex_table(
    list(runs.keys()),
    target_names_to_display_name,
    report_metrics,
    report_metric_names,
    new_dfs,
)


Algorithm            & GMM40 (2d) ELBO          & GMM40 (2d) EUBO          & GMM40 (2d) Sinkhorn      & GMM40 (5d) ELBO          & GMM40 (5d) EUBO          & GMM40 (5d) Sinkhorn      & Funnel (10d) ELBO        & Funnel (10d) EUBO        & Funnel (10d) Sinkhorn    & Many Well (32d) ELBO     & Many Well (32d) EUBO     & Many Well (32d) Sinkhorn
DDS                 
	& -2.36\scriptsize$\pm$0.04 
	& 159.16\scriptsize$\pm$25.17 
	& 595.54\scriptsize$\pm$9.94 
	& -5.16\scriptsize$\pm$0.02 
	& 3054.27\scriptsize$\pm$84.85 
	& 3009.57\scriptsize$\pm$0.82 
	& -0.49\scriptsize$\pm$0.01 
	& 9.67\scriptsize$\pm$1.54 
	& 136.85\scriptsize$\pm$0.84 
	& 160.71\scriptsize$\pm$0.02 
	& 259.08\scriptsize$\pm$6.67 
	& 29.58\scriptsize$\pm$0.02 
\\
LogVar              
	& -2.36\scriptsize$\pm$0.02 
	& 296.63\scriptsize$\pm$46.23 
	& 616.78\scriptsize$\pm$1.96 
	& -5.19\scriptsize$\pm$0.01 
	& 2936.04\scriptsize$\pm$424.35 
	& 3059.83\scriptsize$\pm$99.07 
	& -0.50\scriptsize$\pm$0.01 
	& 8.66\scriptsize$\